In [22]:
import pandas as pd
import numpy as np

In [23]:
flavors = pd.read_csv('flavors_beverages.csv')  
brands = pd.read_csv("brands_soft_drinks.csv")
csd = pd.read_csv("carbonated_soft_drinks.csv")

In [24]:
flavors_list = ['Acai']  # flavors list initialization(ignore Adai entry) 
flavors_single = []   # list used to add double worded flavors into multiple single entries to be used later

brands_list = []      # same thing for brands
brands_single = []

for i in flavors.iloc[:,0]:        ## Looping over flavors to populate above list
    i_new = i.replace(',','')
    i_new = i_new.lower()
    wc = len(i.split())
    if wc<3:                         ## ignoring 3 worded flavors too complex and barely used in real dataset
        flavors_list.append(i_new)
        if wc==2:                      ## spopulating flavors single words
            temp = i_new.split()
            for j in temp:
                flavors_single.append(j)
                
flavors_single = [*set(flavors_single)]       ## Removing duplicates by using a set instead  

In [25]:

## Same process as above but for brands

for i in brands.iloc[:,0]:
    i_new = i.replace(',','')
    i_new = i_new.lower()
    brands_list.append(i_new)
    wc = len(i.split())
    if wc==2:
            temp = i_new.split()
            for j in temp:
                brands_single.append(j)
        

brands_single= [*set(brands_single)] 

## below brands are found in real dataset but differ in brands dataset for example we have fanta but not fantas
## add as needed and found

brands_list.append('fantas')
brands_list.append('cola-coke')
brands_list.append('dr.brown')


In [26]:
## building rest of vocab

# function used to check if word is found in a given list
def check(word, list):
    if word not in list:
        return True
    else:
        return False
        
vocab_rest=[]   ## initialize rest of vocab

for i in csd.iloc[:,0]:
    i_new = i.replace(',','')
    i_new = i_new.replace('(','')
    i_new = i_new.replace(')','')
    i_new = i_new.replace('_',' ')   ## preprocess vocab
    i_new = i_new.lower()    
    words = i_new.split()
    for w in words:           ## check if vocab words belong to brands or flavors to avoid tagging as Outside if so
        if (check(w,flavors_list)) and (check(w,brands_list)) and (check(w,flavors_single)) and (check(w,brands_single)):
            vocab_rest.append(w)    ## add to list if checks out
        
vocab_rest = [*set(vocab_rest)]    ## set for no duplicates      
    

In [27]:
# Initializing random dataset
random_dataset = []


def generate_random_words(a,b,c):
    word_qty = np.random.randint(2, size = (3,2))          ## array to check for random words used 6 binary numbers indicating if 0,1 or 2 words is used in beginning, mid and end of sentence
    word_ind = np.random.randint(len(a)+1, size= (3,2))  ## actual indexes if numbers end up being used 
   
    brand_qty = np.random.randint(2, size = 2)             # array of 2 binary numbers to indicate if 0,1 or 2 brands used [0,0] no brands used, [1,0] or [0,1] only 1, [1,1] 2 brands used
    brand_ind = np.random.randint(len(b)+1, size= 2)    ## array of indexes to be multiplied by above array
    
    flav_qty = np.random.randint(2, size = 2)           ## same thing for flav
    flav_ind = np.random.randint(len(c)+1, size= 2)     # ps the +1 is used to have the 0 index as no value instead of first entry of list
    
    return word_qty*word_ind, brand_qty*brand_ind, flav_qty*flav_ind  ## return the multiplied arrays

def add_brand(ind,exqmple,tags):               ## function used to add brands with tags
    if ind>0:                                  ## easpecially useful for double or more worded brands
        brand = brands_list[ind-1].split()     # not that necessary otherwise
        b_len = len(brand)
        example.append(brand[0])
        tags.append('B-BRA')
        if b_len>1:
            for i in range(b_len-1):
                example.append(brand[i+1])
                tags.append('I-BRA')
def add_flav(ind,example,tags):    ## same function as above for flavors
    if ind>0:
        flav = flavors_list[ind-1].split()
        b_len = len(flav)
        example.append(flav[0])
        tags.append('B-FLA')
        if b_len>1:
            for i in range(b_len-1):
                example.append(flav[i+1])
                tags.append('I-FLA')


    
## main random dataset building function    
for i in range(20000):                       ## 20k examples decided
    example = []       #initializing arrays at every step to clear [previous entries]
    tags = []
    random, brands, flavors = generate_random_words(vocab_rest, brands_list, flavors_list)    ## call matrix function
    for i in random[0]:                     ## add first 0-2 random words using first row of matrix
        if i>0:
            example.append(vocab_rest[i-1])      ## add words
            tags.append('O')                     # add coresponding tags
            
    add_brand(brands[0],example,tags)      # add 0-1 brand depending on matrix with tages using above function
    add_flav(flavors[0],example,tags)     # same for flavs
    
    for i in random[1]:                                     # second set of 0-2 random words
           if i>0:
                example.append(vocab_rest[i-1])
                tags.append('O')
                
    add_brand(brands[1],example,tags)          #second set of brand if not 0
    add_flav(flavors[1],example,tags)          # second flavor if not 0
    
    for i in random[2]:                       # last 0-2 random words
        if i>0:
            example.append(vocab_rest[i-1])
            tags.append('O')
            
    random_dataset.append([example,tags])   ## add examples and tags to dataset
    

    
    
    


In [28]:
supervised_dataset = pd.DataFrame(random_dataset) # create dataframe out of previous obtained dataset

In [29]:
supervised_dataset.to_csv('Supervised_Tagged_Dataset.csv') # write to csv

In [30]:
flavors_list.index('pineapple')


348

In [35]:
#displaying the tagged dataset 
supervised_dataset

,0,1
0,"[12.5o, 12.5, mountian, 24, slrite]","[O, O, O, O, O]"
1,"[flamin, appletiser, drink-medium, sangria, se...","[O, B-BRA, O, B-BRA, I-BRA, O, O]"
2,"[orangina, graham, marshmallow, dto, powerade,...","[B-BRA, B-FLA, I-FLA, O, B-BRA, O]"
3,"[fz, variety, mtdew]","[O, O, O]"
4,"[volta, fantas, orange, pure, mountain, dew, k...","[O, B-BRA, B-FLA, O, B-BRA, I-BRA, B-FLA, O, O]"
...,...,...
19995,"[flavor--shakes/coke, fresca, wink]","[O, B-BRA, B-BRA]"
19996,"[$1.90, carnberry, rc, cola, butter, cream, co...","[O, O, B-BRA, I-BRA, B-FLA, I-FLA, O, O, B-BRA..."
19997,"[rootbeer, almond, chocolate, venom, energy]","[O, B-FLA, I-FLA, B-BRA, I-BRA]"
19998,"[coca-cola, grape, (various)]","[B-BRA, B-FLA, I-FLA]"
